# Databricksを使った(MLFlow)、Anthropic 4.0 sonnect サンプル 

In [ ]:
import os
import mlflow.deployments
from dotenv import load_dotenv

# Only required when running this example outside of a Databricks Notebook

load_dotenv()  # take environment variables

# write to .env

client = mlflow.deployments.get_deploy_client("databricks")
from urllib.request import urlopen, Request
import base64
image_url = os.environ['IMAGE_SAMPLE_URL']
request = Request( image_url, headers={"User-Agent": "Mozilla/5.0"})
image_data = base64.b64encode(urlopen(request).read()).decode("utf-8")
image_format = os.environ['FILE_SUFFIX']
data_url = f"data:image/{os.environ['FILE_SUFFIX']};base64,{image_data}"

In [ ]:
import requests
import IPython.display as Disp

Disp.Image(requests.get(image_url).content)

In [ ]:
prompt_system = {
    "role": "user",
    "content": f"{os.environ['SYSTEM_PROMPT']}"
}
prompt_user = {
            "role": "user",
            "content": [ {
                "type": "text",
                "text": os.environ['PROMPT'].replace("\\n","\n")
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }]
        }
completions_response = client.predict(
    endpoint=os.environ['DATABRICKS_MODEL_NAME'],
    inputs={"messages":[prompt_system, prompt_user]}
)

### Print the response

In [ ]:
print(f"{completions_response.choices[0]['message']['role']}: {completions_response.choices[0]['message']['content']}")
print("Model:", completions_response.model)
print("Usage:")
print("\tPrompt tokens:", completions_response.usage.prompt_tokens)
print("\tCompletion tokens:", completions_response.usage.completion_tokens)
print("\tTotal tokens:", completions_response.usage.total_tokens)

In [ ]:
with open(f"output-use-mlflow-anthropic-{os.environ['DATABRICKS_MODEL_NAME']}.markdown", "w") as o:
    print(completions_response.choices[0]['message']['content'], file=o)